In [1]:
"""
Step6
用戶點擊richmenu，跳出carousel

目標:
學會使用Carousel Template

"""

'\nStep6\n用戶點擊richmenu，跳出carousel\n\n目標:\n學會使用Carousel Template\n\n'

In [2]:
# 區塊1
# 匯入套件

from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, FollowEvent, ImageSendMessage, StickerSendMessage, \
    StickerMessage, TemplateSendMessage, ButtonsTemplate, MessageAction, ConfirmTemplate, CarouselTemplate, \
    CarouselColumn
)

import json
import random

In [3]:
# 區塊2
# 啟用API
# 要先從 secretFile.txt 取出 channelAccessToken 和 channelSecret
# 多增加 linkRichmenuToUser 方法
# 多增加 getProfile 方法

app = Flask(__name__)

secretFile = json.load(open("../secretFile.txt","r"))
richmenuId = secretFile['richmenuId']
channelAccessToken = secretFile['channelAccessToken']
channelSecret = secretFile['channelSecret']
userId = secretFile['userId']

line_bot_api = LineBotApi(channelAccessToken)
handler = WebhookHandler(channelSecret)

def linkRichmenuToUser():
    line_bot_api.link_rich_menu_to_user(userId, richmenuId)
    
def getProfile():

    profile = line_bot_api.get_profile(userId)
    
    print(profile.display_name)
    print(profile.user_id)
    print(profile.picture_url)
    print(profile.status_message)
    
    return profile

In [4]:
# 區塊3
# 設置入口

# 從 Line 的 Webhook URL 接到這裡
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

In [5]:
# 區塊4-1
# 設定導流的事件
# 收到 FollowEvent

@handler.add(FollowEvent)
def Follow_message(event):
    
    # 拿到 getProfile 物件，
    profile = getProfile()
    
    # 取得 display_name屬性
    displayName = profile.display_name
    
    # 拿到 getProfile 物件，取得 picture_url屬性
    pictureUrl = profile.picture_url
    
    # 放入等一下要傳的訊息
    sendMessage = [
        
        # 文字消息1
        TextSendMessage(
            text="哈囉%s，打開下面選單可以看到我們提供的服務喔!" % (displayName)
        ),
        
        # 文字消息2
        TextSendMessage(
            text="喔對了，這是你醜醜的照片"
        ),
        
        # 圖片消息
        ImageSendMessage(
            original_content_url= pictureUrl,
            preview_image_url= pictureUrl
        ),
        
        # 貼圖消息
        StickerSendMessage(
            package_id='11537',
            sticker_id='51626512'
        )
        
    ]
    
    # 發送歡迎詞
    if sendMessage:
        line_bot_api.reply_message(
            event.reply_token,
            sendMessage)
    
    # 綁定richmenu
    linkRichmenuToUser()

In [6]:
# 區塊4-2
# 設定導流的事件
# 收到 MessageEvent
# MessageEvent子分類-StickerMessage


@handler.add(MessageEvent,message = StickerMessage)
def stickerMessage(event):
    
    # 隨機抽取貼圖編號
    stickerId = str(random.randint(51626494,51626533))
    
    
    sendMessage = [
        StickerSendMessage(
            package_id='11537',
            sticker_id=stickerId
        )
    ]
    
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage
    )

In [7]:
# 區塊4-3
# 設定導流的事件
# 收到 MessageEvent
# MessageEvent子分類-TextMessage

@handler.add(MessageEvent,message = TextMessage)
def textMessage(event):
    
    # 將要發送的訊息設為一個List
    sendMessage = []

    # 取出用戶發送的訊息
    message = event.message.text
    
    # 先做第一層判斷，這個訊息是關於甚麼的
    if '[::richmenu::]' in message:
        
        # 接下來再做第二層判斷
        if '各種語錄' in message:
            
            # 製作button template
            buttons_template_message = TemplateSendMessage(
                alt_text='各種語錄',
                template=ButtonsTemplate(
                    title='各種語錄',
                    text='隨便點一個吧',
                    actions=[
                        MessageAction(
                            label='英雄',
                            text='[::template::]英雄',
                        ),
                        MessageAction(
                            label='科學家',
                            text='[::template::]科學家'
                        ),
                    ]
                )
            )
            
            # 將這個 button template 加到要傳送的訊息裡面
            sendMessage.append(buttons_template_message)
            
            
        if '古人語錄' in message:
            
            # 製作button template
            carousel_template_message = TemplateSendMessage(
                alt_text='Carousel template',
                template=CarouselTemplate(
                    columns=[
                        CarouselColumn(
                            thumbnail_image_url='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/mainCarouselEast.jpg?raw=true',
                            title='東方',
                            text='東方古人',
                            actions=[
                                MessageAction(
                                    label='曾參',
                                    text='[::mainCarousel::][::east::]曾參'
                                ),
                                MessageAction(
                                    label='曹操',
                                    text='[::mainCarousel::][::east::]曹操'
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/mainCarouselWest.jpg?raw=true',
                            title='西方',
                            text='西方古人',
                            actions=[
                                MessageAction(
                                    label='哥倫布',
                                    text='[::mainCarousel::][::west::]哥倫布'
                                ),
                                MessageAction(
                                    label='阿基米德',
                                    text='[::mainCarousel::][::west::]阿基米德'
                                )
                            ]
                        )
                    ]
                )
            )
            
            # 將這個 carousel_template_message 加到要傳送的訊息裡面
            sendMessage.append(carousel_template_message) 
            
    # 先做第一層判斷，這個訊息是關於甚麼的
    if '[::template::]' in message:
        
        # 接下來再做第二層判斷
        if '英雄' in message:
            
            # 隨機取得一個亂數
            randNum = str(random.randint(1,5))
            
            # 將亂數加入Url裡
            imageUrl='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/templateHero%s.jpg?raw=true' % (randNum)
        
        if '科學家' in message:
            
            # 隨機取得一個亂數
            randNum = str(random.randint(1,4))
            
            # 將亂數加入Url裡
            imageUrl='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/templateScientist%s.jpg?raw=true' % (randNum)
        
        # 製作 image 訊息
        image_message = ImageSendMessage(
            original_content_url=imageUrl,
            preview_image_url=imageUrl
        )
        
        # 將這個 image 訊息 加到要傳送的訊息裡面
        sendMessage.append(image_message)
        
        
        # 增加確認訊息，詢問用戶是否要離開當前功能
        confirm_template_message = TemplateSendMessage(
            alt_text='Confirm template',
            template=ConfirmTemplate(
                text='要繼續玩嗎?',
                actions=[
                    MessageAction(
                        label='是',
                        text='[::confirmTemplate::][::richmenu::]各種語錄'
                    ),
                    MessageAction(
                        label='否',
                        text='[::confirmTemplate::]離開'
                    )
                ]
            )
        )
        
        # 將這個 confirm template 加到要傳送的訊息裡面
        sendMessage.append(confirm_template_message)
    
    # 如果用戶按了"否"，就離開當前功能
    if '[::confirmTemplate::]離開' in message:
        text_message = TextSendMessage(text='好的，謝謝你的使用')
        
        # 將這個 text message 加到要傳送的訊息裡面
        sendMessage.append(text_message)

    # 如果用戶使用Carousel
    if '[::mainCarousel::]' in message:
        if '[::east::]' in message:
            if '曹操' in message:
                text_message = TextSendMessage(text='寧可我負天下人，不讓天下人負你')
            else:
                text_message = TextSendMessage(text='吾日三省吾身，四省你')
        if '[::west::]' in message:
            if '哥倫布' in message:
                text_message = TextSendMessage(text='我最偉大的成就不是發現新大陸，是發現你')
            else:
                text_message = TextSendMessage(text='給我一個支點，我能翹動你的心')
        sendMessage.append(text_message)

        
    # 傳送訊息
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage
    )

In [8]:
# 區塊5
# 啟動Server

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [29/Apr/2019 11:37:44] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [29/Apr/2019 11:37:53] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [29/Apr/2019 11:37:56] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [29/Apr/2019 11:37:59] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [29/Apr/2019 11:38:02] "POST / HTTP/1.1" 200 -


In [1]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, FollowEvent, ImageSendMessage, StickerSendMessage, \
    StickerMessage, TemplateSendMessage, ButtonsTemplate, MessageAction, ConfirmTemplate, CarouselTemplate, \
    CarouselColumn
)

import json
import requests
import random

app = Flask(__name__)

secretFile = json.load(open("../secretFile.txt","r"))
richmenuId = secretFile['richmenuId']
channelAccessToken = secretFile['channelAccessToken']
channelSecret = secretFile['channelSecret']
userId = secretFile['userId']

line_bot_api = LineBotApi(channelAccessToken)
handler = WebhookHandler(channelSecret)

def getProfile():

    profile = line_bot_api.get_profile(userId)
    
    print(profile.display_name)
    print(profile.user_id)
    print(profile.picture_url)
    print(profile.status_message)
    
    return profile

def linkRichmenuToUser():
    line_bot_api.link_rich_menu_to_user(userId, richmenuId)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(FollowEvent)
def followMessage(event):
    displayName = getProfile()
    
    profile = getProfile()
    
    displayName = profile.display_name
    pictureUrl = profile.picture_url
    
    sendMessage = [
        TextSendMessage(
            text="哈囉%s，打開下面選單可以看到我們提供的服務喔!" % (displayName)
        ),
        
        TextSendMessage(
            text="喔對了，這是你醜醜的照片"
        ),
        
        ImageSendMessage(
            original_content_url= pictureUrl,
            preview_image_url= pictureUrl
        ),
        
        StickerSendMessage(
            package_id='11537',
            sticker_id='51626512'
        )
        
    ]
    
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage
    )
    linkRichmenuToUser()

    
@handler.add(MessageEvent,message = StickerMessage)
def stickerMessage(event):
    
    stickerId = str(random.randint(51626494,51626533))
    
    sendMessage = [
        StickerSendMessage(
            package_id='11537',
            sticker_id=stickerId
        )
    ]
    
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage
    )
    
@handler.add(MessageEvent,message = TextMessage)
def textMessage(event):
    
    sendMessage = []

    message = event.message.text
    
    if '[::richmenu::]' in message:
        if '各種語錄' in message:
            
            # 製作button template
            buttons_template_message = TemplateSendMessage(
                alt_text='各種語錄',
                template=ButtonsTemplate(
                    title='各種語錄',
                    text='隨便點一個吧',
                    actions=[
                        MessageAction(
                            label='英雄',
                            text='[::template::]英雄',
                        ),
                        MessageAction(
                            label='科學家',
                            text='[::template::]科學家'
                        ),
                    ]
                )
            )
            
            # 將這個message加到要傳送的訊息裡面
            sendMessage.append(buttons_template_message)
        
        if '古人語錄' in message:
            carousel_template_message = TemplateSendMessage(
                alt_text='Carousel template',
                template=CarouselTemplate(
                    columns=[
                        CarouselColumn(
                            thumbnail_image_url='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/mainCarouselEast.jpg?raw=true',
                            title='東方',
                            text='東方古人',
                            actions=[
                                MessageAction(
                                    label='曾參',
                                    text='[::mainCarousel::][::east::]曾參'
                                ),
                                MessageAction(
                                    label='曹操',
                                    text='[::mainCarousel::][::east::]曹操'
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/mainCarouselWest.jpg?raw=true',
                            title='西方',
                            text='西方古人',
                            actions=[
                                MessageAction(
                                    label='哥倫布',
                                    text='[::mainCarousel::][::west::]哥倫布'
                                ),
                                MessageAction(
                                    label='阿基米德',
                                    text='[::mainCarousel::][::west::]阿基米德'
                                )
                            ]
                        )
                    ]
                )
            )
            sendMessage.append(carousel_template_message)
        
        
        
    if '[::template::]' in message:
        if '英雄' in message:
            
            randNum = str(random.randint(1,5))
            imageUrl='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/templateHero%s.jpg?raw=true' % (randNum)
            
            image_message = ImageSendMessage(
                original_content_url=imageUrl,
                preview_image_url=imageUrl
            )
            
            sendMessage.append(image_message)
        
        if '科學家' in message:
            
            randNum = str(random.randint(1,4))
            imageUrl='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/templateScientist%s.jpg?raw=true' % (randNum)
            
            image_message = ImageSendMessage(
                original_content_url=imageUrl,
                preview_image_url=imageUrl
            )
            
            sendMessage.append(image_message)
    
        # 增加確認訊息，詢問用戶是否要離開當前功能
        confirm_template_message = TemplateSendMessage(
            alt_text='Confirm template',
            template=ConfirmTemplate(
                text='要繼續玩嗎?',
                actions=[
                    MessageAction(
                        label='是',
                        text='[::confirmTemplate::][::richmenu::]各種語錄'
                    ),
                    MessageAction(
                        label='否',
                        text='[::confirmTemplate::]離開'
                    )
                ]
            )
        )
    
        sendMessage.append(confirm_template_message)
    
    if '[::confirmTemplate::]離開' in message:
        text_message = TextSendMessage(text='好的，謝謝你的使用')
        
        sendMessage.append(text_message)
        
    
    if '[::mainCarousel::]' in message:
        if '[::east::]' in message:
            if '曹操' in message:
                text_message = TextSendMessage(text='寧可我負天下人，不讓天下人負你')
            else:
                text_message = TextSendMessage(text='吾日三省吾身，四省你')
        if '[::west::]' in message:
            if '哥倫布' in message:
                text_message = TextSendMessage(text='我最偉大的成就不是發現新大陸，是發現你')
            else:
                text_message = TextSendMessage(text='給我一個支點，我能翹動你的心')
        sendMessage.append(text_message)
    
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage
    )
    
    
if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [28/Apr/2019 16:33:44] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 16:33:49] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 16:33:55] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 16:33:58] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 16:34:02] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 16:34:06] "POST / HTTP/1.1" 200 -
